# Making a Bayes Classifier from scratch

### Importing libraries


In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
from math import exp
from math import pi


### Reading data

In [2]:
df = pd.read_csv('Dataset1.csv')
ndf = df.to_numpy()
df.head(7)

,x1,x2,y
0,-1.346823,1.229842,1
1,0.607223,-1.060221,3
2,-0.718530,0.948297,1
3,-1.497588,1.264141,1
4,1.217092,1.040403,2
5,0.517683,-1.165068,3
6,-0.893270,-0.713677,0


### Initializing given values

In [3]:
u01 = -1
u02 = -1
u11 = -1
u12 = 1
u21 = 1
u22 = 1
u31 = 1
u32 = -1

var1 = 0.1
var2 = 0.1

probclass0 = 0.1
probclass1 = 0.3
probclass2 = 0.25
probclass3 = 0.35

### Building a classifier function

In [4]:
def Bclassifier(ndf,u1,u2, classprob,va1 ,va2):
        expo =(exp((-(ndf[0] - u1)**2)/(2*va1) + (-(ndf[1] - u2)**2)/(2*va2)))/2*pi*(va1*va2)**(1/2) 
        prob = expo*classprob
        return prob

### Classifing data

In [5]:
predclass = [-1 for i in range(10000)]

for i in range(10000):
    
    maxp = 0
    maxpi = -1
    for j in range(4):
        temp =-10
        match j :
            case 0:
                temp = Bclassifier(ndf[i,:],u01,u02, probclass0,var1,var2)
            case 1:
                temp = Bclassifier(ndf[i,:],u11,u12, probclass1,var1,var2)
            case 2:
                temp = Bclassifier(ndf[i,:],u21,u22, probclass2,var1,var2)
            case 3:
                temp = Bclassifier(ndf[i,:],u31,u32, probclass3,var1,var2)
        
        if temp> maxp:
            maxp =temp
            maxpi =j
            
    predclass[i] = maxpi



### Probability Of Error

In [6]:
accu = accuracy_score(ndf[:,2] , predclass)
POE = 1-accu
print(POE)

0.0020000000000000018


# PART 2


### Calculating Priori Probability of training set

In [7]:
classcount0 = classcount1 = classcount2 = classcount3 = 0

for i in range(5000):
    
    match ndf[i,2]:
        case 0:
            classcount0 +=1
        case 1:
            classcount1 +=1
        case 2:
            classcount2 +=1
        case 3: 
            classcount3 +=1

            
            
prior0 =classcount0/5000            
prior1 =classcount1/5000 
prior2 =classcount2/5000
prior3 =classcount3/5000


print("Calculating Priori Probability for first 5000 elements as training set")
print("Priori probability of class 0:", prior0)
print("Priori probability of class 1:", prior1)
print("Priori probability of class 2:", prior2)
print("Priori probability of class 3:", prior3)

Calculating Priori Probability for first 5000 elements as training set
Priori probability of class 0: 0.1034
Priori probability of class 1: 0.2926
Priori probability of class 2: 0.256
Priori probability of class 3: 0.348


### Calculating Varience for training set

In [8]:
def varience(ni):
    sumvar =0
    
    if ni ==0:
        u0 = u1= -1
        u2 = u3 = 1
    elif ni ==1:
        u0 = u3= -1
        u2 = u1 = 1         
        
    for i in range(5000):

        match ndf[i,2]:
            case 0:
                n = (ndf[i,ni]-u0)**2
                sumvar += n
            case 1:
                n = (ndf[i,ni]-u1)**2
                sumvar += n
            case 2:
                n = (ndf[i,ni]-u2)**2
                sumvar += n
            case 3:
                n = (ndf[i,ni]-u3)**2
                sumvar += n
                
    newvar1 = sumvar/5000
    print(newvar1)
    return newvar1

In [9]:
newv1 = varience(0)
newv2 = varience(1)

0.09939782901427276
0.09822951195635882


### Classifing data for test set ie 5000 to 10000 elements

### Calculating POE using information given.

In [10]:
predclass = [-1 for i in range(5000)]

for i in range(5000):
    
    maxp = 0
    maxpi = -1
    for j in range(4):
        temp =-10
        match j :
            case 0:
                temp = Bclassifier(ndf[i+5000,:],u01,u02, probclass0,var1,var2)
            case 1:
                temp = Bclassifier(ndf[i+5000,:],u11,u12, probclass1,var1,var2)
            case 2:
                temp = Bclassifier(ndf[i+5000,:],u21,u22, probclass2,var1,var2)
            case 3:
                temp = Bclassifier(ndf[i+5000,:],u31,u32, probclass3,var1,var2)
        
        if temp> maxp:
            maxp =temp
            maxpi =j
            
    predclass[i] = maxpi

    
accu = accuracy_score(ndf[-5000:,2] , predclass)
POE = 1-accu
print(POE)

0.0020000000000000018


### Calculating POE using obtained vales

In [11]:
predclass = [-1 for i in range(5000)]

for i in range(5000):
    
    maxp = 0
    maxpi = -1
    for j in range(4):
        temp =-10
        match j :
            case 0:
                temp = Bclassifier(ndf[i+5000,:],u01,u02, prior0,newv1,newv2)
            case 1:
                temp = Bclassifier(ndf[i+5000,:],u11,u12, prior1,newv1,newv2)
            case 2:
                temp = Bclassifier(ndf[i+5000,:],u21,u22, prior2,newv1,newv2)
            case 3:
                temp = Bclassifier(ndf[i+5000,:],u31,u32, prior3,newv1,newv2)
        
        if temp> maxp:
            maxp =temp
            maxpi =j
            
    predclass[i] = maxpi


### Predicting POE for test set

In [12]:
accu = accuracy_score(ndf[-5000:,2] , predclass)
POE = 1-accu
print(POE)

0.0018000000000000238
